# Docker DB서버와 연동

In [1]:
import cx_Oracle
import pandas as pd
import json
from pyarrow import csv
import numpy as np

In [2]:
data_path ='../data/'
# Json 파일 읽어오기
def read_json(jsonPath ,mod = 'r',encoding ='utf-8'):
    with open(jsonPath, mod, encoding = encoding) as common:
        config = json.load(common)
    return config
config = read_json(data_path + "json/config.json")

## Oracle 접속

In [4]:
CONN_INFO = {
        'NAME': 'XEPDB1',
        'USER': 'iitp',
        'PASSWORD': 'iitp',
        # 'HOST': '172.16.5.231',
        'HOST': '172.16.0.25',
        'PORT': '11521',
}
CONN_STR = '{USER}/{PASSWORD}@{HOST}:{PORT}/{NAME}'.format(**CONN_INFO)
conn = cx_Oracle.connect(CONN_STR)
cursor = conn.cursor()

In [5]:

def clean_colName(colNames,col_name = '시도,년도',val_cnt = ':1,:2',tmp_cnt = 3):
    tabel_cols = '시도 varchar2(38), 년도 varchar2(38)'
    for cols in colNames:
        cols = cols.replace(' (%)', '')
        cols = cols.replace(' (명)', '')
        cols = cols.replace('[명]', '')
        cols = cols.replace('[백분율]', '')
        cols = cols.replace(')', '')
        cols = cols.replace('(', '_')
        cols = cols.replace(' ', '_')
        cols = cols.replace('1', '')
        col_name+= f',{cols}'
        tabel_cols+= f', {cols} varchar2(38)'
        val_cnt += f',:{tmp_cnt}'

        tmp_cnt+=1 
    return col_name,val_cnt,tabel_cols
    

# 필요 Table Create 및 

## 명세 테이블 생성 및 삽입

In [75]:
# 시도 코드 명세 테이블 생성
try :
    cursor.execute("""
        DROP TABLE sido_code 
    """)
except:
    pass    
finally:   
    cursor.execute("""
        CREATE TABLE sido_code ( 
            code    NUMBER(4)	NOT NULL,
            eng_name    VARCHAR2(20) NOT NULL, 
            kor_name    VARCHAR2(6) primary key
        )
    """) 
    # 값 삽입
    df = csv.read_csv(config['data_path']+ 'csv/specification/sido_code.csv').to_pandas()
    sql = """insert into sido_code(code,eng_name,kor_name)
             values (:1,:2,:3)"""         
    val = list(map(tuple,df.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()

In [156]:
# 범죄 명세 테이블 생성

try :
    cursor.execute("""
        DROP TABLE crime_Specification 
    """)
except:
    pass
        
finally:
    cursor.execute("""
        CREATE TABLE crime_Specification ( 
            code    NUMBER(4)	NOT NULL,
            crime_name    VARCHAR2(30) primary key
        )
    """)
    # 값 삽입
    df = csv.read_csv(config['data_path']+ 'csv/specification/crime_Specification.csv').to_pandas()
    sql = """insert into crime_Specification(code,crime_name)
             values (:1,:2)"""         
    val = list(map(tuple,df.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()

## corr를 이용하여 추출한 칼럼리스트 

In [187]:
sido = pd.read_sql("select * from sido_code",con=conn) 
crime_Specification = pd.read_sql("select * from crime_Specification",con=conn) 
corr_csv = csv.read_csv(config['data_path']+ 'csv/corr/colList.csv').to_pandas()
corr_csv['시도'] = corr_csv['시도'].map(sido.set_index('KOR_NAME').to_dict()['CODE'])
corr_csv['범죄'] = corr_csv['범죄'].map(crime_Specification.set_index('CRIME_NAME').to_dict()['CODE'])
aa = corr_csv.dropna()
aa

,시도,범죄,칼럼리스트
0,42,살인,"['경제활동인구', '취업자', '고용률 (%)', '실업자', '1인당 지역내총생..."
1,42,성폭력,"['경제활동인구', '취업자', '실업자', '실업률 (%)', '1인당 지역내총생..."
2,42,폭행,"['이혼율', '경제활동인구', '취업자', '고용률 (%)', '실업자', '1인..."
3,42,상해,"['실업자', '실업률 (%)', '경찰청 소속 경찰관 수']"
4,42,협박,"['경제활동인구', '취업자', '고용률 (%)', '실업자', '실업률 (%)',..."
...,...,...,...
177,36,약취와 유인,"['1인당 지역내총생산', '1인당 민간소비', 'PC방', '음식점', '유흥가'..."
178,36,주거침입,"['1인당 지역내총생산', '1인당 지역총소득', '1인당 개인소득', '1인당 민..."
179,36,유기,"['1인당 지역내총생산', '1인당 민간소비', 'PC방', '음식점', '유흥가'..."
180,36,공무방해,"['1인당 지역내총생산', '1인당 지역총소득', '유흥가', '향정신성의약품 청구..."


In [ ]:
# Top5 Corr 테이블 생성
corr_csv = csv.read_csv(config['data_path']+ 'csv/cleaned/Top_Five_Cols.csv').to_pandas()
corr_csv['범죄'] = corr_csv['범죄'].map(crime_Specification.set_index('CRIME_NAME').to_dict()['CODE'])

try :
    cursor.execute("""
        DROP TABLE topFiveCols
    """)
except:
    pass        
finally:
    corr_csv = csv.read_csv(config['data_path']+ 'csv/cleaned/Top_Five_Cols.csv').to_pandas()
    corr_csv['범죄'] = corr_csv['범죄'].map(crime_Specification.set_index('CRIME_NAME').to_dict()['CODE'])
    
    cursor.execute(f'create table topFiveCols ( code NUMBER(4)	PRIMARY KEY, colList    VARCHAR2(150) NOT NULL  )')     
    sql = 'insert into topFiveCols( code, colList) values (:1,:2)' 
    corr_csv = corr_csv.astype(str).fillna('').drop_duplicates()
    val = list(map(tuple,corr_csv.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()

## 정제된 데이터 테이블 생성 및 삽입

In [6]:
# 시도 코드 맵핑
sido = pd.read_sql("select * from sido_code",con=conn) 
merged_csv = csv.read_csv(config['data_path']+ 'csv/cleaned/crime_indexed.csv').to_pandas()
merged_csv['시도'] = merged_csv['시도'].map(sido.set_index('KOR_NAME').to_dict()['CODE'])
crime = merged_csv.iloc[:,:4]
crime = crime.astype(str).fillna('')

In [141]:
# 범죄 테이블 생성

try :
    cursor.execute("""
        DROP TABLE crime 
    """)
except:
    pass        
finally:
    cursor.execute("""
        create table crime ( 
            시도    varchar2(38) ,
            년도    varchar2(38) ,
            범죄종류    varchar2(38),
            범죄수    varchar2(38)
        )
    """)    
    sql = """insert into crime(시도 ,년도 ,범죄종류 ,범죄수)
             values (:1,:2,:3,:4)"""     
    val = list(map(tuple,crime.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()
    

In [142]:
# 종속변수 테이블 생성
raw_ind =csv.read_csv(config['data_path']+ 'csv/cleaned/crime_merged.csv').to_pandas()
raw_ind = pd.concat([raw_ind[raw_ind.columns[:2]],raw_ind[raw_ind.columns[15:]]],axis = 1)
raw_ind['시도'] = raw_ind['시도'].map(sido.set_index('KOR_NAME').to_dict()['CODE'])

try :
    cursor.execute("""
        DROP TABLE independent_val 
    """)
except:
    pass        
finally:
    col_name , val_cnt,tabel_cols= clean_colName(raw_ind.columns[2:])
    cursor.execute(f'create table independent_val ( {tabel_cols} )')     
    sql = f'insert into independent_val( {col_name} ) values ( {val_cnt} )' 
    indi_val = raw_ind.astype(str).fillna('').drop_duplicates()
    val = list(map(tuple,indi_val.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()
    

In [15]:
na_filled_zero =csv.read_csv(config['data_path']+ 'csv/cleaned/independent/to_zero.csv').to_pandas()
na_filled_zero['시도'] = na_filled_zero['시도'].map(sido.set_index('KOR_NAME').to_dict()['CODE'])
na_filled_zero.년도.unique()

array([2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
       2024, 2025, 2026, 2027, 2028, 2029, 2030], dtype=int64)

In [18]:
#  결측갑 처리후 예측한 종속변수
na_filled_zero =csv.read_csv(config['data_path']+ 'csv/cleaned/independent/to_zero.csv').to_pandas()
na_filled_zero['시도'] = na_filled_zero['시도'].map(sido.set_index('KOR_NAME').to_dict()['CODE'])

na_filled_mean =csv.read_csv(config['data_path']+ 'csv/cleaned/independent/to_mean.csv').to_pandas()
na_filled_mean['시도'] = na_filled_mean['시도'].map(sido.set_index('KOR_NAME').to_dict()['CODE'])

# 0으로 대체
try :
    cursor.execute("""
        DROP TABLE independent_zero
    """)
except:
    pass        
finally:
    col_name , val_cnt,tabel_cols= clean_colName(na_filled_zero.columns[2:])
    cursor.execute(f'create table independent_zero ( {tabel_cols} )')     
    sql = f'insert into independent_zero( {col_name} ) values ( {val_cnt} )' 
    indi_val = na_filled_zero.astype(str).fillna('').drop_duplicates()
    val = list(map(tuple,indi_val.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()
    
# 종속변수 테이블 생성
# Mean으로 대체
try :
    cursor.execute("""
        DROP TABLE independent_mean
    """)
except:
    pass        
finally:
    col_name , val_cnt,tabel_cols= clean_colName(na_filled_mean.columns[2:])
    cursor.execute(f'create table independent_mean ( {tabel_cols} )')     
    sql = f'insert into independent_mean( {col_name} ) values ( {val_cnt} )' 
    indi_val = na_filled_mean.astype(str).fillna('').drop_duplicates()
    val = list(map(tuple,indi_val.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()
    

# 최종 예측값 DB에 삽입

In [10]:
finalPredict = csv.read_csv(config['data_path']+ 'csv/cleaned/crime_predicted_xgboost_mean_scaled.csv').to_pandas()
finalPredict

# 종속변수 테이블 생성
# Mean으로 대체
try :
    cursor.execute("""
        DROP TABLE final_predict
    """)
except:
    pass        
finally:
    col_name , val_cnt,tabel_cols = clean_colName(finalPredict.columns[2:])
    cursor.execute(f'create table final_predict ( {tabel_cols} )')     
    sql = f'insert into final_predict( {col_name} ) values ( {val_cnt} )' 
    indi_val = finalPredict.astype(str).fillna('').drop_duplicates()
    val = list(map(tuple,indi_val.values.tolist()))
    cursor.executemany(sql,val)
    conn.commit()